# День 06. Создание модели данных с нуля   

Вы решили реализовать проект “Умный дом”.   

Давайте рассмотрим следующие типы датчиков для конкретной квартиры дома:  
- Датчик для управления жалюзи шторами в квартире
- Датчик движения в квартире
- Датчик протечки воды в квартире
- Датчик открытия двери / окна в квартире
- Датчик дыма в квартире

Указанные датчики формируют экосистему квартиры, которая может состоять из множества датчиков разного типа.   

Каждый датчик характеризуется:     
- Уникальным номером (идентификатором) внутри экосистем всех квартир, характеризующийся набором цифр и буквенных символов (например ‘дат№1_№123’)  
- Типом датчика - строковое значение с указанием Типа датчика, которые перечислены ниже (например ‘Датчик движения’)  
- Является ли он включенным / выключенным в экосистеме квартиры (значение Истина / Ложь)  
- Датой ввода в эксплуатацию (с указанием дня / месяца / года)  
- Номером квартиры, где он установлен - строковое значение (например ‘3405’)  


Каждый датчик посылает сигнал о своем состоянии **каждые 10 секунд в базу данных.** Возможны следующие значения состояний по датчикам:   
- **Датчик движения** отсылает два состояния (0 - движение не обнаружено, 1 - движение обнаружено)  
- **Датчик протечки воды** отсылает два состояния (0 - протечка не обнаружена, 1- протечка обнаружена)  
- **Датчик открытия двери** отсылает два состояния (0 - дверь / окно не открыто, 1- дверь / окно открыто)
- **Датчик открытия окна** отсылает два состояния (0 - дверь / окно не открыто, 1- дверь / окно открыто)
- **Датчик для управления жалюзи** отсылает два состояния (0 - жалюзи / шторы закрыты, 1- жалюзи / шторы открыты)
- **Датчик для управления шторами** отсылает два состояния (0 - жалюзи / шторы закрыты, 1- жалюзи / шторы открыты)
- **Датчик дыма** отсылает два состояния (0 - задымление не обнаружено, 1 -  задымление обнаружено)  

## Задача  


1. Создать физическую модель данных для описания зарегистрированных датчиков внутри экосистем всех квартир на основании типа датчика.   
- Не забудьте указать PRIMARY KEY   
- Не забудьте указать FOREIGN KEY (если в этом есть необходимость)  
- Не забудьте указать UNIQUE KEY для внутреннего номера датчика  


2. Создать физическую модель данных оперативной таблицы для хранения значений показателей всех датчиков со всех квартир дома по указанной дате ( с учетом дня / месяца / года / часов / минут / секунд). Обратите внимание что здесь можно использовать паттерн EAV (Entity-Attribute-Value) модели данных.   
- Не забудьте указать PRIMARY KEY  
- Не забудьте указать FOREIGN KEY  
- Не забудьте указать UNIQUE KEY, для предотвращения сохранения данных от одного и того же датчика в одно и тоже время  
- Не забудьте указать правило (CHECK CONSTRAINT) что значение любого датчика принимает только два значения 0 / 1.   

3. Зарегистрировать 5 датчиков разного типа в вашу модель данных для квартиры № ‘123-А’ с датой ввода в эксплуатацию 25 сентября 2022 года.

4. Создать 20 показаний по разным типам датчиков для квартиры № ‘123-А’ для оперативных данных (с интервалом времени в 10 секунд для конкретного типа датчика) и с произвольным значением состояния датчика 0 или 1.  
5. Написать SQL запрос для получения списка значений отосланных состояний за указанный промежуток по любому датчику, отсортированных по дате (времени) получения в обратном порядке.   

Пример результата вывода представлен ниже

| внутренний номер датчика | тип датчика | значение состояния датчика | дата состояния датчика | номер квартиры |
| ------ | ------ | ------ | ------ | ------ |
| 123 | Датчик движения | 0 | 2022-09-25 20:17:51.000000 | 123-А |
| 123 | Датчик движения | 1 | 2022-09-25 20:17:41.000000 | 123-А |
| 123 | Датчик движения | 1 | 2022-09-25 20:17:31.000000 | 123-А |
| 123 | Датчик движения | 1 | 2022-09-25 20:17:21.000000 | 123-А |

In [20]:
#импорт необходимых библиотек
from datetime import datetime as dt
from datetime import timedelta
import random
import itertools

from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine, MetaData, Table, select, func, text
from sqlalchemy.ext.automap import automap_base

import pandas as pd
import time

## Идея проекта
Создание таблицы detectors, в которой будут содержаться данные о датчиках и таблицы с подключенной trigger-функцией detectors_logs, которая каждые 10 секунд будет отслеживать изменение состояния датчиков.

Таблица **detectors** содержит следующие столбы:
- *id* - идентификатор датчика
- *flat* - номер квартиры
- *detector_type* - тип датчика
- *status* - состояние датчика (включен или выключен)
- *start_date* - дата ввода в эксплуатацию

Таблица **detectors_logs** содержит следующие столбы:
- *time_of_update* - время обновления состояния датчика
- *id* - идентификатор датчика
- *flat* - номер квартиры
- *detector_type* - тип датчика
- *status* - состояние датчика (включен или выключен)
- *start_date* - дата ввода в эксплуатацию

In [21]:
#подключение к локальной БД
USERNAME = "postgres"  # заменить на свой логин

connection_string = f"postgresql+psycopg2://{USERNAME}:@localhost:5432/{USERNAME}"
engine = create_engine(connection_string)
Session = sessionmaker(engine)

In [37]:
#создание таблицы detectors и detectors_logs из файла с SQL-запросом. 
#Если таблицы уже созданы, код выдаст ошибку
f = open('SmartHome.sql', 'r', encoding='utf-8')
creation_query = f.read()
f.close()

with Session() as session:
    session.execute(text(creation_query))
    session.commit()
    
meta = MetaData()
meta.reflect(engine)

detectors = Table("detectors", meta, autoload_with=engine)

Base = automap_base()
Base.prepare(autoload_with=engine)

Detector = Base.classes.detectors

NameError: name 'UniqueViolation' is not defined

In [38]:
#достаем из detectors идентификатор датчика
ids_query = "SELECT DISTINCT(id) FROM detectors"
with Session() as session:
    result = session.execute(text(ids_query))
det_ids = pd.DataFrame.from_records(result)[0]
det_ids

0    7123
1    4123
2    3123
3    6123
4    5123
5    2123
6    1123
Name: 0, dtype: int64

In [39]:
#генерируем состояния датчиков с интервалом в 10 секунд
random.seed(42)
for i in range(20):
    for det_id in det_ids:
        det_state = random.choice([0,1])
        update_query = f"""
            UPDATE detectors
            SET status = '{det_state}'
            WHERE id = {det_id};"""
        with Session() as session:
            session.execute(text(update_query))
            session.commit()
    time.sleep(10)    
        

KeyboardInterrupt: 

In [ ]:
#пишем функцию, которая на вход принимает идентификатор датчика и возвращает датафрейм с изменениями состояния датчика
def report(detector_id):
    report_list = []
    report_query = f"""
    SELECT id "внутренний номер датчика", detector_type "тип датчика", status "значение состояния датчика", 
        time_of_update "дата состояния датчика", flat "номер квартиры" 
    FROM detectors_logs
    WHERE id = {detector_id}
    ORDER BY time_of_update DESC"""
    with Session() as session:
        result = session.execute(text(report_query))
    detector_data = pd.DataFrame.from_records(result, columns =["внутренний номер датчика", "тип датчика", "значение состояния датчика", 
"дата состояния датчика", "номер квартиры"])    
    return detector_data

Преобразовываем результат запроса в DataFrame.

In [40]:
#тестируем функцию
detector_report = report(7123)

detector_report

,внутренний номер датчика,тип датчика,значение состояния датчика,дата состояния датчика,номер квартиры
0,7123,Датчик дыма,False,2023-04-25 22:43:17.764327,123
1,7123,Датчик дыма,False,2023-04-25 22:43:07.739498,123
2,7123,Датчик дыма,True,2023-04-25 22:33:17.641376,123
3,7123,Датчик дыма,False,2023-04-25 22:33:07.609271,123
4,7123,Датчик дыма,True,2023-04-25 22:32:57.571627,123
5,7123,Датчик дыма,True,2023-04-25 22:32:47.533629,123
6,7123,Датчик дыма,True,2023-04-25 22:32:37.503210,123
7,7123,Датчик дыма,True,2023-04-25 22:32:27.470426,123
8,7123,Датчик дыма,False,2023-04-25 22:32:17.450491,123
9,7123,Датчик дыма,True,2023-04-25 22:32:07.420144,123
